<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_thoughtless_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_thoughtless_ai.py

1. Synthetic dataset: 6 inputs → 3 targets
2. Float32 normalization
3. MLP with Dropout for MC‐Dropout uncertainty
4. Physics-informed residual enforcing toy laws
5. Combined loss: MSE + residual
6. Training: AdamW, LR scheduler, grad clipping, early stopping
7. Checkpointing & plotting: loss, parity, uncertainty
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# 1. Synthetic Dataset
# -----------------------------------------------------------------------------
class ThoughtlessDataset(Dataset):
    def __init__(self, n_samples=5000, seed=123):
        np.random.seed(seed)
        # Inputs: cognitive_flux, intelligence_emerge, pre_thought, idle_rational, meta_cog, latent_pot
        CF = np.random.uniform(0.0, 2.0, (n_samples,1))
        IE = np.random.uniform(0.1, 5.0, (n_samples,1))
        PT = np.random.uniform(-2.0, 2.0,(n_samples,1))
        IR = np.random.uniform(0.0, 3.0,(n_samples,1))
        MC = np.random.uniform(-1.0, 1.0,(n_samples,1))
        LP = np.random.uniform(0.5, 2.0,(n_samples,1))
        X_raw = np.hstack([CF, IE, PT, IR, MC, LP]).astype(np.float32)

        # Toy ground truth:
        # pre_coh   = CF * IE / (|PT| + eps)
        # post_res  = IR * sin(MC)
        # undef_eq  = LP * (pre_coh + post_res)
        eps = 1e-6
        pre_coh  = CF * IE / (np.abs(PT) + eps)
        post_res = IR * np.sin(MC)
        undef_eq = LP * (pre_coh + post_res)
        Y_raw = np.hstack([pre_coh, post_res, undef_eq]).astype(np.float32)

        # Add small Gaussian noise
        noise = 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape).astype(np.float32)
        Y_raw += noise

        # Compute and store normalization stats
        self.X_mean = X_raw.mean(0).astype(np.float32)
        self.X_std  = X_raw.std(0).astype(np.float32) + eps
        self.Y_mean = Y_raw.mean(0).astype(np.float32)
        self.Y_std  = Y_raw.std(0).astype(np.float32) + eps

        # Normalize
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# -----------------------------------------------------------------------------
# 2. Model with Dropout
# -----------------------------------------------------------------------------
class ThoughtlessAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, p_drop=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p_drop),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# -----------------------------------------------------------------------------
# 3. Physics‐Informed Residual
# -----------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    CF, IE, PT, IR, MC, LP = X_den.t()
    eps = 1e-6

    pre_coh_t   = CF * IE / (PT.abs() + eps)
    post_res_t  = IR * torch.sin(MC)
    undef_eq_t  = LP * (pre_coh_t + post_res_t)

    Y_true = torch.stack([pre_coh_t, post_res_t, undef_eq_t], dim=1)
    Y_norm = (Y_true - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Y_norm)

# -----------------------------------------------------------------------------
# 4. Combined Loss
# -----------------------------------------------------------------------------
def total_loss(pred, target, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, target)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# -----------------------------------------------------------------------------
# 5. MC‐Dropout Uncertainty
# -----------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()  # dropout active
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    stacked = torch.stack(preds)
    return stacked.mean(0), stacked.std(0)

# -----------------------------------------------------------------------------
# 6. Training Loop
# -----------------------------------------------------------------------------
def train(model, tr_dl, va_dl, stats, device,
          lr=1e-3, wd=1e-5, lam=1.0,
          epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)

    best_val = float('inf')
    wait     = 0
    history  = {'train': [], 'val': []}

    for ep in range(1, epochs+1):
        # Train
        model.train()
        train_loss = 0.0
        for Xb, Yb in tr_dl:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lam)

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * Xb.size(0)
        train_loss /= len(tr_dl.dataset)

        # Validate
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for Xv, Yv in va_dl:
                Xv, Yv = Xv.to(device), Yv.to(device)
                pred = model(Xv)
                loss, _, _ = total_loss(pred, Yv, Xv, stats, lam)
                val_loss += loss.item() * Xv.size(0)
        val_loss /= len(va_dl.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {ep:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Checkpoint
        if val_loss < best_val - 1e-6:
            best_val = val_loss
            wait = 0
            torch.save(model.state_dict(), "best_thoughtless_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load("best_thoughtless_ai.pth", map_location=device))
    return history

# -----------------------------------------------------------------------------
# 7. Visualization
# -----------------------------------------------------------------------------
def plot_history(h):
    plt.figure()
    plt.plot(h['train'], label='Train')
    plt.plot(h['val'],   label='Val')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_parity(model, ds, stats, device):
    model.eval()
    X = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        P = model(X).cpu().numpy()
    Y_true = (ds.Y * ds.Y_std + ds.Y_mean)
    P_denorm = P * ds.Y_std + ds.Y_mean

    plt.figure(figsize=(6,6))
    plt.scatter(Y_true.ravel(), P_denorm.ravel(), s=4, alpha=0.5)
    m, M = Y_true.min(), Y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.show()

def plot_uncertainty(model, ds, stats, device):
    X = torch.from_numpy(ds.X).to(device)
    _, std = mc_dropout_predict(model, X, T=50)
    u = std[:,0].cpu().numpy()
    plt.hist(u, bins=30, color='teal')
    plt.xlabel('Stddev of Pre-Thought Pred.')
    plt.show()

# -----------------------------------------------------------------------------
# 8. Main Execution
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset and stats
    ds = ThoughtlessDataset(n_samples=5000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std':  torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }

    # Split and loaders
    val_count = int(len(ds) * 0.2)
    tr_ds, va_ds = random_split(ds, [len(ds)-val_count, val_count])
    tr_dl = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=256, shuffle=False)

    # Train and visualize
    model   = ThoughtlessAI()
    history = train(model, tr_dl, va_dl, stats, device)

    plot_history(history)
    plot_parity(model, ds, stats, device)
    plot_uncertainty(model, ds, stats, device)